In [4]:
from Bio import SeqIO,  AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import Phylo
from Bio import SeqIO
from Bio.Phylo.TreeConstruction import DistanceCalculator

from scipy.stats import ttest_1samp

import pandas as pd 
import re
import matplotlib as plt


/var/folders/q8/_1lfzn8160v3_gfsx1j1f_vw0000gn/T/ipykernel_28341/2372515124.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:



#Expression using the re function to find a set of strings that match it: this tells python to match this "[breed="
#The bracets () tells python what we want to retrive, '.' matches any character, '*' mathces any sequence of characters after 
#'?' tells python to lazy match ie match as little characters as possible 
breed_search = re.compile(r'\[breed=(.*?)\]')

#set our list to store breeds 
breed_database = []

with open('./dog_breeds.fa', 'r') as file:
    for record in SeqIO.parse(file, 'fasta'): #Open fasta file
        breed_retrive = breed_search.search(record.description)  #Search the description in each sequence and get the breed
        if breed_retrive: #If statment to checks whether it can find the pattern 
            breed = breed_retrive.group(1) #If it does then we capture the second part of out search (works as a index) ie '.*?' this w
        record.annotations['breed'] = breed #This is tellign python for the extracted breeds we what to put them as an annotation in our record 
        breed_database.append(record) #Appends the data, SeIO.parse automatically pulls thre record.ID and seq to append, (by setting breed to record annotations we also append the breed to the list )


#Open mystery seqeunce file -> I have set this to my query seqeunce 
query_sequence = SeqIO.read('./mystery.fa', 'fasta')

#We want to determine the breed so what I will do is set the breed to 'Mystery' for now
query_sequence.annotations['breed'] = 'mystery'

#I want to combine the query_sequence with the breed_database so I can so a MSA so I need to make a list containinf the query_sequence 
query_sequence_data = [query_sequence]

#Combine the lists together using '+'
combined_data = breed_database + query_sequence_data

# Conduct MSA using the MultipleSeqAlignment fucntion of biopython
alignment = MultipleSeqAlignment(combined_data)

#Print data to check if geneID, and breed and seqeunces have been extracted correctly and saved 

# for record in alignment:
#     print(record.id, record.annotations['breed']) 
#     print(record.seq)




In [6]:
#Define a fucntion to calulate the sequence identity 

def sequence_identity_calculation (alignment, query_sequence): #Set our inputs 
    alignment_length = alignment.get_alignment_length() #get the lenths of each alignment 

    #Make list to store alignment score 
    identity_scores = []
    
    #For loop to loop through the aligment 
    for record in alignment:
        #Retriving matching nucleotides (count the number of matchning nucleotides in the ) 
        #1 for i in range means iterate over each postion in our alignment sequence
        match_count = sum(1 for i in range(alignment_length) if record[i] == query_sequence[i]) #if it matches the postion in the query_seqeunce then count it as 1 
        #Divide the match_count by the aligmnet length then multiply by 100 to get the identitiy_percentage 
        identity_percentage = (match_count / alignment_length) * 100 
        #append the record.id, record breed and identity percentage to a list)
        identity_scores.append((record.id, [record.annotations['breed']], identity_percentage))


    return identity_scores





In [8]:
identity_scores = sequence_identity_calculation(alignment, query_sequence)

In [9]:
print(identity_scores)

[('gb|CM023446.1|', ['boxer'], 99.7609799820735), ('gb|MW916043.1|', ['Portuguese Warren dog, small size, smooth hair'], 99.71915147893637), ('gb|MW916029.1|', ['Azores Cattle dog'], 99.7131759784882), ('gb|MW916019.1|', ['Aidi'], 99.71915147893637), ('gb|MW916055.1|', ['Portuguese Sheepdog'], 99.82073498655512), ('gb|MW916054.1|', ['Portuguese Sheepdog'], 99.82073498655512), ('gb|MW916050.1|', ['Estrela Mountain dog'], 99.7131759784882), ('gb|MW916042.1|', ['Portuguese Warren dog, small size, weired hair'], 99.84463698834777), ('gb|MW916039.1|', ['Portuguese Warren dog, medium size, weired hair'], 99.82073498655512), ('gb|KU290784.1|', ['Mixed breed'], 99.72512697938453), ('gb|KU290444.1|', ['Camp dog'], 99.7131759784882), ('gb|MW916071.1|', ['Portuguese Pointing Dog-long hair'], 99.81475948610696), ('gb|MW916070.1|', ['Portuguese Pointing Dog-long hair'], 99.81475948610696), ('gb|MW916069.1|', ['Portuguese Pointing Dog-long hair'], 99.81475948610696), ('gb|MW916068.1|', ['Portuguese 

In [34]:
for record, (record_id, breed, identity) in zip(alignment, identity_scores):
    print(f"GenBank accession: {record_id}, Breed: {breed}, Identity: {identity}%")\
    
    print(record.seq)

GenBank accession: gb|CM023446.1|, Breed: ['boxer'], Identity: 99.7609799820735%
GTTAATGTAGCTTAATTAATAAAGCAAGGCACTGAAAATGCCAAGATGAGTCGCACGACTCCATAAACATAAAGGTTTGGTCCTAGCCTTCCTATTAGTTTTTAGTAGACTTACACATGCAAGCCTCCACGCCCCAGTGAGAATGCCCTTAAAATCACCAGTGATCTAAAGGAGCAGGTATCAAGCACACTCTTAAGTAGCTCATAACACCTTGCTAAGCCACACCCCCACGGGATACAGCAGTGATAAAAATTAAGCCATAAACGAAAGTTTGACTAAGCCATACTAAATAGGGTTGGTAAATTTCGTGCCAGCCACCGCGGTCATACGATTAACCCAAACTAATAGGCCTACGGCGTAAAGCGTGTTCAAGATACTTTTACACTAAAGTTAAAACTTAACTAAGCCGTAAAAAGCTACAGTTATCATAAAATAAACCACGAAAGTGACTTTATAATAATCTGACTACACGATAGCTAAGACCCAAACTGGGATTAGATACCCCACTATGCTTAGCCCTAAACATAGATAATTTTACAACAAAATAATTCGCCAGAGGACTACTAGCAATAGCTTAAAACTCAAAGGACTTGGCGGTGCTTTATATCCCTCTAGAGGAGCCTGTTCTATAATCGATAAACCCCGATAAACCTCACCACCTTTCGCTAATTCAGTCTATATACCGCCATCTTCAGCAAACCCTCAAAAGGTAGAACAGTAAGCACAATCATTTTACATAAAAAAGTTAGGTCAAGGTGTAACTTATGAGGTGGGAAGAAATGGGCTACATTTTCTACCCAAGAACATTTCACGAATGTTTTTATGAAATTAAAAACTGAAGGAGGATTTAGTAGTAAATTAAGAATAGAGAGCTTAATTGAATAGGGCCATGAAGCACGCACACACCGCCCGTCACCCT

In [42]:


data = []
for record, (record_id, breed, identity) in zip(alignment, identity_scores):
    record_data = {
       "record_id_breed": str(record_id) + str(breed),
        "breed": breed,
        "identity_(%)": identity,
        "sequence": str(record.seq) 
    }
    data.append(record_data)
    
breed_match_data = pd.DataFrame(data)



print(breed_match_data)


                                      record_id_breed  \
0                             gb|CM023446.1|['boxer']   
1   gb|MW916043.1|['Portuguese Warren dog, small s...   
2                 gb|MW916029.1|['Azores Cattle dog']   
3                              gb|MW916019.1|['Aidi']   
4               gb|MW916055.1|['Portuguese Sheepdog']   
..                                                ...   
95                      gb|KU290523.1|['Mixed breed']   
96                         gb|KU290442.1|['Camp dog']   
97             gb|MW916065.1|['Algarve Barrocal Dog']   
98                   gb|MW916075.1|['EpagneulBreton']   
99                          gb|KM061522.1|['mystery']   

                                               breed  identity_(%)  \
0                                            [boxer]     99.760980   
1   [Portuguese Warren dog, small size, smooth hair]     99.719151   
2                                [Azores Cattle dog]     99.713176   
3                                  

In [43]:
print(breed_match_data)


                                      record_id_breed  \
0                             gb|CM023446.1|['boxer']   
1   gb|MW916043.1|['Portuguese Warren dog, small s...   
2                 gb|MW916029.1|['Azores Cattle dog']   
3                              gb|MW916019.1|['Aidi']   
4               gb|MW916055.1|['Portuguese Sheepdog']   
..                                                ...   
95                      gb|KU290523.1|['Mixed breed']   
96                         gb|KU290442.1|['Camp dog']   
97             gb|MW916065.1|['Algarve Barrocal Dog']   
98                   gb|MW916075.1|['EpagneulBreton']   
99                          gb|KM061522.1|['mystery']   

                                               breed  identity_(%)  \
0                                            [boxer]     99.760980   
1   [Portuguese Warren dog, small size, smooth hair]     99.719151   
2                                [Azores Cattle dog]     99.713176   
3                                  

In [44]:
#Finding max identity score -> need to filter data, we can sort it by 
sorted_data = breed_match_data.sort_values(by='identity_(%)', ascending=False)
#print(sorted_data)

#Now we can breed match by pulling the second row 
breed_match = sorted_data.iloc[1]
print(breed_match)


record_id_breed           gb|AY656744.1|['English Springer Spaniel']
breed                                     [English Springer Spaniel]
identity_(%)                                               99.856588
sequence           GTTAATGTAGCTTAATTAATAAAGCAAGGCACTGAAAATGCCAAGA...
Name: 93, dtype: object


In [45]:
from scipy.stats import ttest_1samp


identity_percentages = breed_match_data['identity_(%)']

t_statistic, p_value = ttest_1samp(identity_percentages, 100)

print("T-statistic:", t_statistic)
print("P-value:", p_value)



T-statistic: -44.942430358683694
P-value: 1.1362428772677818e-67


In [16]:
from Bio.Phylo.TreeConstruction import DistanceCalculator

In [17]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import AlignIO

# 
calculator = DistanceCalculator('identity')
dist_matrix = calculator.get_distance(alignment)
print(dist_matrix)
print(type(dist_matrix))





gb|CM023446.1|  0.000000
gb|MW916043.1|  0.001315    0.000000
gb|MW916029.1|  0.001374    0.000060    0.000000
gb|MW916019.1|  0.001434    0.000359    0.000418    0.000000
gb|MW916055.1|  0.001494    0.001255    0.001315    0.001255    0.000000
gb|MW916054.1|  0.001494    0.001255    0.001315    0.001255    0.000000    0.000000
gb|MW916050.1|  0.001494    0.000418    0.000478    0.000179    0.001315    0.001315    0.000000
gb|MW916042.1|  0.001494    0.001374    0.001434    0.001374    0.000359    0.000359    0.001434    0.000000
gb|MW916039.1|  0.001494    0.001374    0.001434    0.001374    0.000598    0.000598    0.001434    0.000359    0.000000
gb|KU290784.1|  0.001494    0.000418    0.000418    0.000538    0.001195    0.001195    0.000598    0.001315    0.001434    0.000000
gb|KU290444.1|  0.001494    0.000299    0.000299    0.000657    0.001315    0.001315    0.000717    0.001434    0.001554    0.000120    0.000000
gb|MW916071.1|  0.001554    0.001434    0.001494    0.001315    0

In [18]:
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
constructor = DistanceTreeConstructor()
UPGMATree = constructor.upgma(dist_matrix)

In [27]:
from Bio import Phylo
import matplotlib.pyplot as plt



scaling_factor = 2.0
for clade in UPGMATree.find_clades():
    if clade.branch_length:
        clade.branch_length *= scaling_factor


fig, ax = plt.subplots(figsize=(60, 400))  #Used to adjust the figure size since there is a lot and wont fit into the standard size
Phylo.draw(UPGMATree, axes=ax, do_show=False)


plt.show()